<a href="https://colab.research.google.com/github/nralibera/PAr_135/blob/main/fault_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import random as rd
from tqdm import tqdm


## **I. Préparation des outils pour générer les données**
On a besoin de quelques fonctions afin de générer le tableau contenant les données d'entrainement du modèle. On souhaite notemment deux tableaux: le premier `matrix_data` contenant les matrices (3D avec shape=(n,m,p)) et le deuxième `label` contenant les fautes qui correspondent aux matrices (1D avec shape=(p)). Par exemple, la matrice `matrix_data(:,:,15)` correspond à la faute de type `label(15)`.

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
x_size_matrix=55
y_size_matrix=108 
def build_matrix(matrix_line):
    matrix_line= [x.replace('\n','' ) for x in matrix_line] 
    matrix = np.array([[int(i) for i in x] for x in matrix_line])
    matrix= np.reshape(matrix,(x_size_matrix,y_size_matrix,1))
    return matrix

In [ ]:
def build_data (filename,fault_list):
    matrix_file= open("/content/drive/My Drive/PAR/"+filename,"r")
    matrix_lines = matrix_file.readlines()
    matrix_file.close()
    
    matrix_lines = [line.replace("\n","") for line in matrix_lines]
    
    
    #initialisation
    matrix_data = build_matrix(matrix_lines[1:56])
    label_data = np.array([fault_list.index(matrix_lines[0])])
    
    for i in tqdm(range(56,len(matrix_lines),56)):
        label_data = np.concatenate([label_data,np.array([fault_list.index(matrix_lines[i])])])
        matrix = build_matrix(matrix_lines[i+1:i+56])
        matrix_data=np.concatenate([matrix_data,matrix],axis=2)
    
    return matrix_data,label_data

## **II. Contructions des données pour l'entrainement du modèle :**
Dans cette partie, nous allons construire les tableaux de données avec les fonctions définies ci-dessus. Nous allons, dans un premier tableau, construire la base de données avec les matrices des fautes permanentes ainsi que les fautes intermittentes. Le deuxième tableau contiendra les labels de chaque matrice.

In [ ]:
#building the right_matrix (in txt format)
right_matrix_file=open('/content/drive/My Drive/PAR/right_matrix.txt',"r")
right_matrix = build_matrix(right_matrix_file.readlines())
right_matrix_file.close()

In [ ]:
#building permanent_fault(in txt format)
fault_list_file = open('/content/drive/My Drive/PAR/fault_matrix_file2.txt',"r")
fault_list=fault_list_file.readlines()
fault_list_file.close()
fault_list = [fault_list[i].replace("\n","") for i in range(0,len(fault_list),56)]
permanent_fault_matrix,permanent_fault_label= build_data("fault_matrix_file2.txt",fault_list)

100%|██████████| 3608/3608 [01:43<00:00, 34.84it/s]


In [ ]:
#building intermittent_fault matrix (in txt format)
print((intermittent_fault1_label))
fault_list_file2 = open('/content/drive/My Drive/PAR/equivalent_fault.txt',"r")
fault_list2=fault_list_file2.readlines()
fault_list_file2.close()
fault_list2 = [i.replace("\n","") for i in fault_list2]

intermittent_fault1_matrix,intermittent_fault1_label = build_data("intermittent_fault.txt",fault_list2)
intermittent_fault2_matrix,intermittent_fault2_label = build_data("intermittent_fault2.txt",fault_list2)


[   0    0    0 ... 1367 1367 1367]


In [13]:
#Concatenating all faults in one matrix and label
nb_array=3609
matrix_file=np.load('/content/drive/My Drive/PAR/intermittent_fault_file_compressed/intermittent_fault_array0.npz')
matrix_all_faults=matrix_file['matrix']
labels_all_faults=matrix_file['label']
for k in range(1,4):
  compressed_file_name="intermittent_fault_array"+str(k)
  matrix_file= np.load("/content/drive/My Drive/PAR/intermittent_fault_file_compressed/"+compressed_file_name+ ".npz")
  matrix_fault_i=matrix_file['matrix']
  matrix_label_i=matrix_file['label']
  matrix_all_faults = np.concatenate([matrix_all_faults,matrix_fault_i],axis=2)
  labels_all_faults = np.concatenate([labels_all_faults,matrix_label_i])
np.savez_compressed('matrix_all_faults_compressed',matrix=matrix_all_faults,label=labels_all_faults)